In [1]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Module relatif à Gurobi
from gurobipy import *
import json

In [2]:
# Opening JSON file
with open('toy_instance.json') as json_file:
    data = json.load(json_file)

In [41]:
# Implémentation Python
n_staff = len(data["staff"])
n_job = len(data["jobs"])
n_days = data["horizon"]
n_qual = len(data["qualifications"])

eps = 10**(-3)
# Instanciation du modèle
m = Model("PL modelling using matrix")

# Création d'un vecteur de 3 variables continues
v = m.addMVar(shape=(n_staff, n_job, n_days, n_qual), vtype=GRB.BINARY)

# qualification personnnel
for i in range(n_staff):
    for q in range(n_qual):
        if not(data["qualifications"][q] in data["staff"][i]["qualifications"]):
            m.addConstr(v[i,:,:,q].sum() == 0)


# Contrainte d'unicité
for i in range(n_staff):
    for t in range(n_days):
        m.addConstr(v[i,:,t,:].sum() <= 1)
        

# Contrainte de congés
for i in range(n_staff):
    for t in range(n_days):
        if t+1 in data["staff"][i]["vacations"]:
            m.addConstr(v[i,:,t,:].sum() == 0)

# Contrainte de couverture
for p in range(n_job):
    for q in range(n_qual):
        qual = data["qualifications"][q]
        if qual in data["jobs"][p]["working_days_per_qualification"].keys():
            m.addConstr(v[:,p,:,q].sum() <= data["jobs"][p]["working_days_per_qualification"][qual])
            
        
        else:
            m.addConstr(v[:,p,:,q].sum() <= 0)

# def job_real
job_real = m.addMVar(shape=(n_job), vtype=GRB.BINARY)
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(v[:,p,:,:].sum() >= n_qual_need  - (n_qual_need+ eps)*(1-job_real[p]))
    m.addConstr(v[:,p,:,:].sum() <= n_qual_need + (n_qual_need+ eps)*(job_real[p]))

# def end_date
end_date = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days)
b_end_date = m.addMVar(shape=(n_job, n_days), vtype=GRB.BINARY)
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(end_date[p] <= n_days)
    for t in range(n_days):
        m.addConstr(v[:,p,:t+1,:].sum()>= n_qual_need -eps - (n_days + eps) * (1 - b_end_date[p,t]))
        m.addConstr(v[:,p,:t+1,:].sum()<= n_qual_need - eps + (n_days + eps) * b_end_date[p,t])
        
        m.addConstr(end_date[p] <= t * b_end_date[p,t] + n_days * (1 - b_end_date[p,t]))
        m.addConstr(end_date[p] >= (t + eps) * (1 - b_end_date[p,t]))

        
# def start_date
start_date = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days)
b_start_date = m.addMVar(shape=(n_job, n_days), vtype=GRB.BINARY)
for p in range(n_job):
    n_qual_need = sum(data["jobs"][p]["working_days_per_qualification"].values())
    m.addConstr(start_date[p] <= n_days)
    for t in range(n_days):
        m.addConstr(v[:,p,:t+1,:].sum()>= eps - (n_days + eps) * (1 - b_start_date[p,t]))
        m.addConstr(v[:,p,:t+1,:].sum()<= eps + (n_days + eps) * b_start_date[p,t])
        
        m.addConstr(start_date[p] <= t * b_start_date[p,t] + n_days * (1 - b_start_date[p,t]))
        m.addConstr(start_date[p] >= (t + eps) * (1 - b_start_date[p,t]))

# def number of project per employee
project_per_employee = m.addMVar(shape=(n_staff), vtype=GRB.INTEGER, lb=0, ub=n_job)
b_project_per_employee = m.addMVar(shape=(n_staff, n_job), vtype=GRB.BINARY)
for i in range(n_staff):
    m.addConstr(project_per_employee[i] <= n_job)
    for p in range(n_job):
        m.addConstr(v[i,p,:,:].sum()>= eps - (n_days + eps) * (1 - b_project_per_employee[i,p]))
        m.addConstr(v[i,p,:,:].sum()<= eps + (n_days + eps) * b_project_per_employee[i,p])
        
    m.addConstr(project_per_employee[i] == b_project_per_employee[i,:].sum())

max_project_per_employee = m.addMVar(shape=(1), vtype=GRB.INTEGER)
m.addGenConstrMax(max_project_per_employee, list(project_per_employee))

# def has_penality
has_penality = m.addMVar(shape=(n_job), vtype=GRB.BINARY)
penality = m.addMVar(shape=(n_job), vtype=GRB.INTEGER, lb=0, ub=n_days*data["jobs"][0]["daily_penalty"])
for p in range(n_job):
    due_date = data["jobs"][p]["due_date"] -1
    m.addConstr(end_date[p] >= due_date + eps - (n_days + eps) * (1 - has_penality[p]))
    m.addConstr(end_date[p] <= due_date + (n_days + eps) * (has_penality[p]))
    
    m.addConstr(penality[p] == has_penality[p] * data["jobs"][p]["daily_penalty"] * (end_date[p] - due_date))


# def duration
duration = m.addMVar(shape=(n_job), vtype=GRB.INTEGER)
m.addConstr(duration == end_date - start_date)

max_duration = m.addMVar(shape=(1), vtype=GRB.INTEGER)
m.addGenConstrMax(max_duration, list(duration))

<gurobi.GenConstr *Awaiting Model Update*>

In [42]:
gain = np.zeros(n_job)
for p in range(n_job):
    gain[p] = data["jobs"][p]["gain"]

vec_for_sum = job_real * (gain - penality)

In [43]:
m.setObjective(vec_for_sum.sum(), GRB.MAXIMIZE)

In [44]:
m.params.outputflag = 1
# maj
m.update()
# Affichage en mode texte du PL
print(m.display())

# Résolution du PL
m.optimize()


Maximize
20.0 C225 + 15.0 C226 + 15.0 C227 + 20.0 C228 + 10.0 C229 + [ -1.0 C225 * C314 +
-1.0 C226 * C315 + -1.0 C227 * C316 + -1.0 C228 * C317 + -1.0 C229 * C318 ]
Subject To
R0: C77 + C80 + C83 + C86 + C89 + C92 + C95 + C98 + C101 + C104 + C107 + C110 + C113 +
 C116 + C119 + C122 + C125 + C128 + C131 + C134 + C137 + C140 + C143 + C146 + C149 = 0
R1: C150 + C153 + C156 + C159 + C162 + C165 + C168 + C171 + C174 + C177 + C180 + C183 +
C186 + C189 + C192 + C195 + C198 + C201 + C204 + C207 + C210 + C213 + C216 + C219 + C222
 = 0
R2: C151 + C154 + C157 + C160 + C163 + C166 + C169 + C172 + C175 + C178 + C181 + C184 +
C187 + C190 + C193 + C196 + C199 + C202 + C205 + C208 + C211 + C214 + C217 + C220 + C223
 = 0
R3: C0 + C1 + C2 + C15 + C16 + C17 + C30 + C31 + C32 + C45 + C46 + C47 + C60 + C61 +
 C62 <= 1
R4: C3 + C4 + C5 + C18 + C19 + C20 + C33 + C34 + C35 + C48 + C49 + C50 + C63 + C64 +
 C65 <= 1
R5: C6 + C7 + C8 + C21 + C22 + C23 + C36 + C37 + C38 + C51 + C52 + C53 + C66 + C67 +
 C68 <= 1


 C237 <= 2.999
  R56: C230 + 3.0 C237 <= 5
  R57: C230 + 2.001 C237 >= 2.001
R58: C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9 + C10 + C11 + C75 + C76 + C77 +
C78 + C79 + C80 + C81 + C82 + C83 + C84 + C85 + C86 + C150 + C151 + C152 + C153 + C154 +
 C155 + C156 + C157 + C158 + C159 + C160 + C161 + -5.001 C238 >= -2.002
R59: C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9 + C10 + C11 + C75 + C76 + C77 +
C78 + C79 + C80 + C81 + C82 + C83 + C84 + C85 + C86 + C150 + C151 + C152 + C153 + C154 +
 C155 + C156 + C157 + C158 + C159 + C160 + C161 + -5.001 C238 <= 2.999
  R60: C230 + 2.0 C238 <= 5
  R61: C230 + 3.001 C238 >= 3.001
R62: C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9 + C10 + C11 + C12 + C13 + C14 +
C75 + C76 + C77 + C78 + C79 + C80 + C81 + C82 + C83 + C84 + C85 + C86 + C87 + C88 + C89
+ C150 + C151 + C152 + C153 + C154 + C155 + C156 + C157 + C158 + C159 + C160 + C161 +
 C162 + C163 + C164 + -5.001 C239 >= -2.002
R63: C0 + C1 + C2 + C3 + C4 + C5 + C6 + C7 + C8 + C9 + C10 + C11 + 

C131 + C132 + C133 + C134 + C195 + C196 + C197 + C198 + C199 + C200 + C201 + C202 + C203
 + C204 + C205 + C206 + C207 + C208 + C209 + -5.001 C254 <= 2.999
  R127: C233 + C254 <= 5
  R128: C233 + 4.001 C254 >= 4.001
  R129: C234 <= 5
  R130: C60 + C61 + C62 + C135 + C136 + C137 + C210 + C211 + C212 + -5.001 C255 >= -3.002
  R131: C60 + C61 + C62 + C135 + C136 + C137 + C210 + C211 + C212 + -5.001 C255 <= 1.999
  R132: C234 + 5.0 C255 <= 5
  R133: C234 + 0.001 C255 >= 0.001
R134: C60 + C61 + C62 + C63 + C64 + C65 + C135 + C136 + C137 + C138 + C139 + C140 +
 C210 + C211 + C212 + C213 + C214 + C215 + -5.001 C256 >= -3.002
R135: C60 + C61 + C62 + C63 + C64 + C65 + C135 + C136 + C137 + C138 + C139 + C140 +
 C210 + C211 + C212 + C213 + C214 + C215 + -5.001 C256 <= 1.999
  R136: C234 + 4.0 C256 <= 5
  R137: C234 + 1.001 C256 >= 1.001
R138: C60 + C61 + C62 + C63 + C64 + C65 + C66 + C67 + C68 + C135 + C136 + C137 + C138 +
C139 + C140 + C141 + C142 + C143 + C210 + C211 + C212 + C213 + C214 + C215 

  R203: C262 + 3.0 C277 <= 5
  R204: C262 + 2.001 C277 >= 2.001
R205: C30 + C31 + C32 + C33 + C34 + C35 + C36 + C37 + C38 + C39 + C40 + C41 + C105 +
C106 + C107 + C108 + C109 + C110 + C111 + C112 + C113 + C114 + C115 + C116 + C180 + C181
 + C182 + C183 + C184 + C185 + C186 + C187 + C188 + C189 + C190 + C191 + -5.001 C278 >= -5
R206: C30 + C31 + C32 + C33 + C34 + C35 + C36 + C37 + C38 + C39 + C40 + C41 + C105 +
C106 + C107 + C108 + C109 + C110 + C111 + C112 + C113 + C114 + C115 + C116 + C180 + C181
+ C182 + C183 + C184 + C185 + C186 + C187 + C188 + C189 + C190 + C191 + -5.001 C278 <=
 0.001
  R207: C262 + 2.0 C278 <= 5
  R208: C262 + 3.001 C278 >= 3.001
R209: C30 + C31 + C32 + C33 + C34 + C35 + C36 + C37 + C38 + C39 + C40 + C41 + C42 + C43
+ C44 + C105 + C106 + C107 + C108 + C109 + C110 + C111 + C112 + C113 + C114 + C115 +
C116 + C117 + C118 + C119 + C180 + C181 + C182 + C183 + C184 + C185 + C186 + C187 + C188
 + C189 + C190 + C191 + C192 + C193 + C194 + -5.001 C279 >= -5
R210: C30 + C3

 + C132 + C133 + C134 + -5.001 C301 >= -5
R275: C120 + C121 + C122 + C123 + C124 + C125 + C126 + C127 + C128 + C129 + C130 + C131
 + C132 + C133 + C134 + -5.001 C301 <= 0.001
R276: C135 + C136 + C137 + C138 + C139 + C140 + C141 + C142 + C143 + C144 + C145 + C146
 + C147 + C148 + C149 + -5.001 C302 >= -5
R277: C135 + C136 + C137 + C138 + C139 + C140 + C141 + C142 + C143 + C144 + C145 + C146
 + C147 + C148 + C149 + -5.001 C302 <= 0.001
  R278: C291 + -1.0 C298 + -1.0 C299 + -1.0 C300 + -1.0 C301 + -1.0 C302 = 0
  R279: C292 <= 5
R280: C150 + C151 + C152 + C153 + C154 + C155 + C156 + C157 + C158 + C159 + C160 + C161
 + C162 + C163 + C164 + -5.001 C303 >= -5
R281: C150 + C151 + C152 + C153 + C154 + C155 + C156 + C157 + C158 + C159 + C160 + C161
 + C162 + C163 + C164 + -5.001 C303 <= 0.001
R282: C165 + C166 + C167 + C168 + C169 + C170 + C171 + C172 + C173 + C174 + C175 + C176
 + C177 + C178 + C179 + -5.001 C304 >= -5
R283: C165 + C166 + C167 + C168 + C169 + C170 + C171 + C172 + C173 + C174 

     0     0   66.00000    0    7   65.00000   66.00000  1.54%     -    0s
     0     0   66.00000    0   16   65.00000   66.00000  1.54%     -    0s
     0     0   66.00000    0   19   65.00000   66.00000  1.54%     -    0s

Cutting planes:
  Learned: 4
  Cover: 3
  Clique: 3
  MIR: 16
  GUB cover: 7
  Zero half: 1
  Mod-K: 1
  RLT: 1
  Relax-and-lift: 5

Explored 1 nodes (1069 simplex iterations) in 0.53 seconds (0.05 work units)
Thread count was 4 (of 4 available processors)

Solution count 4: 65 64 59 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 6.500000000000e+01, best bound 6.500000000000e+01, gap 0.0000%


In [45]:
project_per_employee.X

array([3., 2., 4.])

In [46]:
max_project_per_employee.X

array([4.])

In [47]:
job_real.X

array([1., 0., 1., 1., 1.])

In [48]:
penality.X

array([0., 9., 0., 0., 0.])

In [49]:
has_penality.X

array([0., 1., 0., 0., 0.])

In [50]:
start_date.X

array([1., 3., 1., 0., 4.])

In [51]:
end_date.X

array([2., 5., 3., 2., 4.])

In [52]:
duration.X

array([1., 2., 2., 2., 0.])

In [53]:
max_duration.X

array([2.])

In [20]:
data

{'horizon': 5,
 'qualifications': ['A', 'B', 'C'],
 'staff': [{'name': 'Olivia',
   'qualifications': ['A', 'B', 'C'],
   'vacations': []},
  {'name': 'Liam', 'qualifications': ['A', 'B'], 'vacations': [1]},
  {'name': 'Emma', 'qualifications': ['C'], 'vacations': [2]}],
 'jobs': [{'name': 'Job1',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 1, 'C': 1}},
  {'name': 'Job2',
   'gain': 15,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'B': 2}},
  {'name': 'Job3',
   'gain': 15,
   'due_date': 4,
   'daily_penalty': 3,
   'working_days_per_qualification': {'A': 1, 'C': 2}},
  {'name': 'Job4',
   'gain': 20,
   'due_date': 3,
   'daily_penalty': 3,
   'working_days_per_qualification': {'B': 2, 'C': 1}},
  {'name': 'Job5',
   'gain': 10,
   'due_date': 5,
   'daily_penalty': 3,
   'working_days_per_qualification': {'C': 2}}]}